In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
from mofdscribe.chemistry._fragment import get_node_atoms, get_floating_indices, get_bbs_from_indices, fragment
from mofdscribe.utils.structure_graph import get_structure_graph

from pymatgen.core import IStructure

In [17]:
s = IStructure.from_file('/Users/kevinmaikjablonka/git/kjappelbaum/mofdscribe/tests/test_files/HKUST-1.cif')

In [32]:
s.composition

Comp: Cu48 H96 C288 O192

In [18]:
sg = get_structure_graph(s, 'jmolnn')

In [28]:
linker_fragments, node_fragments = fragment(sg)

In [31]:
len(node_fragments[1])

36

In [36]:
import networkx as nx

In [70]:
def knbrs(G, start, k):
    nbrs = set([start])
    for l in range(k):
        nbrs = set((nbr for n in nbrs for nbr in G[n]))
    return nbrs

In [90]:
nx.descendants_at_distance(sg.graph, 90, 3)

set()

In [89]:
sg.structure[90]

PeriodicSite: H (3.1612, 20.3368, 10.0103) [0.1200, 0.7720, 0.3800]

In [41]:
from collections import defaultdict
from mofdscribe.utils.aggregators import AGGREGATORS

In [65]:

def compute_racs(start_indices: Iterable[int], structure_graph: StructureGraph, properties: Tuple[str], scope: int, property_aggregations: Tuple[str], corr_aggregations: Tuple[str], part_name: str=''):
    racs = defaultdict(lambda: defaultdict(list))
    for start_atom in start_indices: 
        neighbors = nx.descendants_at_distance(structure_graph.graph, start_atom, scope)
        
        # We only branch if there are actually neighbors scope many bonds away ...
        if len(neighbors) > 0:
            site = structure_graph.structure[start_atom]
            for neighbor in neighbors:
                n = structure_graph.structure[neighbor]
                for prop in properties:
                    if prop == 1:
                        p0 = 1
                        p1 = 1
                    else:
                        p0 = getattr(site.specie, prop)
                        p1 = getattr(n.specie, prop)

                    for agg in property_aggregations:
                        agg_func = AGGREGATORS[agg]
                        racs[prop][agg].append(agg_func((p0, p1)))

    aggregated_racs = {}
    
    for property_name, property_values in racs.items():
        for aggregation_name, aggregation_values in property_values.items():
            for corr_agg in corr_aggregations:
                agg_func = AGGREGATORS[corr_agg]
                name = f'racs_{part_name}_{property_name}_{scope}_{aggregation_name}_{corr_agg}'
                aggregated_racs[name] = agg_func(aggregation_values)

    return aggregated_racs


In [66]:
compute_racs(linker_fragments[2][0], sg, ('X', 'electron_affinity', 1, 'atomic_mass'), 2, ('avg', 'product', 'diff'), ('sum',))

{'racs__X_2_avg_sum': 5.99,
 'racs__X_2_product_sum': 17.543999999999997,
 'racs__X_2_diff_sum': 1.7800000000000002,
 'racs__electron_affinity_2_avg_sum': 2.723218912,
 'racs__electron_affinity_2_product_sum': 3.6881617753906877,
 'racs__electron_affinity_2_diff_sum': 0.397983376,
 'racs__1_2_avg_sum': 2.0,
 'racs__1_2_product_sum': 2,
 'racs__1_2_diff_sum': 0,
 'racs__atomic_mass_2_avg_sum': 28.0101,
 'racs__atomic_mass_2_product_sum': 384.32798715999996,
 'racs__atomic_mass_2_diff_sum': 7.977399999999999}